In [1]:
from rdflib import Dataset, URIRef
import rdflib.plugins.sparql

from rdflib import Graph
from io import BytesIO
import pandas as pd
import owlrl

rdflib.plugins.sparql.SPARQL_LOAD_GRAPHS = False
rdflib.plugins.sparql.SPARQL_DEFAULT_GRAPH_UNION = False

In [2]:
# Reading the ontology and instances into a graph
g = Graph()

# Reading the ontology and instances into a graph
g.parse('./TBox-foot-infos.ttl')
g.parse('./mapping-output.ttl')

 # Apply the reasoner
reasoner = owlrl.DeductiveClosure(owlrl.RDFS_Semantics)
reasoner.expand(g)

# Let's write the file
# Graphs are mutable
g.serialize(destination="data-full.ttl", format="ttl")

<Graph identifier=N751f8f6c063f44778d32e224ba42f60d (<class 'rdflib.graph.Graph'>)>

In [3]:
res = g.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  SELECT DISTINCT ?type WHERE {
    ?x rdf:type ?type .
  }
""")

res_as_csv = res.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,type
0,http://www.w3.org/2002/07/owl#Ontology
1,http://www.w3.org/2002/07/owl#AnnotationProperty
2,http://www.w3.org/2000/01/rdf-schema#Datatype
3,http://www.w3.org/2002/07/owl#ObjectProperty
4,http://www.w3.org/2002/07/owl#AsymmetricProperty
...,...
63,_:nd642e5db924d4b73b2f7f5fc94140e57b21
64,http://www.w3.org/2000/01/rdf-schema#Literal
65,_:nd642e5db924d4b73b2f7f5fc94140e57b10
66,_:nd642e5db924d4b73b2f7f5fc94140e57b2


In [4]:
len(g)

22977

1. This query finds all the cities in the RDF dataset.

In [5]:
res = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT ?city ?cityName WHERE {
  ?city a foot:Location; 
  foot:cityName ?cityName.
}
""")

res_as_csv = res.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,city,cityName
0,http://www.example.org/foot-infos/resources/ci...,Mexico City
1,http://www.example.org/foot-infos/resources/ci...,San José
2,http://www.example.org/foot-infos/resources/ci...,Nouméa
3,http://www.example.org/foot-infos/resources/ci...,Amsterdam
4,http://www.example.org/foot-infos/resources/ci...,Stockholm
...,...,...
94,http://www.example.org/foot-infos/resources/ci...,Toronto
95,http://www.example.org/foot-infos/resources/ci...,Mexico City
96,http://www.example.org/foot-infos/resources/ci...,Washington D.C.
97,http://www.example.org/foot-infos/resources/ci...,Buenos Aires


2. This query selects the female player, her first name, her last name and her age, making sure she is over 25 in the RDF dataset. 

In [6]:
res = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT ?firstName ?lastName ?age ?genre ?teamName WHERE {
  ?player foot:playsFor ?team.
  ?player a foot:PlayerIdentity;
          foot:playerFirstName ?firstName;
          foot:playerLastName ?lastName;
          foot:playerDateOfBirth ?dateOfBirth;
          foot:playerGenre ?genre .
  ?team a foot:Team; 
        foot:teamName ?teamName.
  BIND(year(now()) - year(?dateOfBirth) AS ?age)
  FILTER (?age > 25 && ?genre = false)
}
""")

res_as_csv = res.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,firstName,lastName,age,genre,teamName
0,Emma,Smith,32,False,Istanbul Eagles
1,Olivia,Martini,29,False,Conakry Cobras
2,Emma,Garcia,29,False,Copenhagen Crusaders
3,Cristiane,Rozeira,39,False,Quito Quicksilvers
4,Chloe,Johnson,28,False,Bamako Bulldogs
5,Emily,Taylor,31,False,Caracas Cougars
6,Emma,Garcia,32,False,Montevideo Mavericks
7,Sarah,Brown,33,False,Nairobi Lions
8,Sophia,Kim,28,False,Luanda Leopards
9,Sam,Kerr,31,False,Stockholm Strikers


3. The query below retrieves information about football players, including their identity, team, playing statistics, as well as details of their place of birth and country of origin. It also filters out players who have at least one assist.

In [7]:
res = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT ?firstName ?playerGenre ?teamName ?assists ?goals ?statistic_date WHERE {

  ?player foot:playsFor ?team.
  ?player a foot:PlayerIdentity;
           foot:playerFirstName ?firstName;
           foot:playerGenre ?playerGenre.
  ?team a foot:Team;
        foot:teamName ?teamName.
        
  ?statistics foot:gottenBy ?player;
              a foot:PlayerFairPlayIndex;
              foot:goals ?goals; 
              foot:assists ?assists;
              foot:playerFairPlayDate ?statistic_date. 
  FILTER (?playerGenre = false || ?playerGenre = true)
}
""")

res_as_csv = res.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,firstName,playerGenre,teamName,assists,goals,statistic_date
0,Daniel,True,Georgetown Gladiators,2,1,2023-06-03
1,Daniel,True,Georgetown Gladiators,2,true,2023-06-03
2,Daniel,True,Georgetown Gladiators,2,1,2023-06-03
3,Hector,True,Paramaribo Panthers,0,2,2023-06-09
4,Hector,True,Paramaribo Panthers,false,2,2023-06-09
...,...,...,...,...,...,...
81,Sophie,False,San Salvador Scorpions,1,false,2023-06-04
82,Sophie,False,San Salvador Scorpions,true,0,2023-06-04
83,Sophie,False,San Salvador Scorpions,true,false,2023-06-04
84,Sophie,False,San Salvador Scorpions,1,0,2023-06-04


In [8]:
res = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>

SELECT ?personFirstName ?personLastName ?dateOfBirth ?genre ?birthLocationName ?countryName ?qualificationZoneName ?personType
WHERE {
  {
    ?person a foot:PlayerIdentity;
            foot:playerIsBornIn ?birthLocation.
    ?person foot:playerFirstName ?personFirstName;
            foot:playerLastName ?personLastName;
            foot:playerDateOfBirth ?dateOfBirth;
            foot:playerGenre ?genre.
    ?birthLocation a foot:Location;
                   foot:cityName ?birthLocationName.
    ?person foot:playerComesFrom ?country.
    ?country foot:countryName ?countryName.
    ?country foot:isIncludedIn ?qualificationZone.
    ?qualificationZone foot:qualificationZoneName ?qualificationZoneName.    
    FILTER (?genre = false || ?genre = true)
    BIND("Player" AS ?personType).
  }
  UNION
  {
    ?person a foot:RefereeIdentity;
            foot:refereeIsBornIn ?birthLocation.
    ?person foot:refereeFirstName ?personFirstName;
            foot:refereeLastName ?personLastName;
            foot:refereeDateOfBirth ?dateOfBirth;
            foot:refereeGenre ?genre.
    ?birthLocation a foot:Location;
                   foot:cityName ?birthLocationName.
    ?person foot:refereeComesFrom ?country.
    ?country foot:countryName ?countryName.
    ?country foot:isIncludedIn ?qualificationZone.
    ?qualificationZone foot:qualificationZoneName ?qualificationZoneName.  
    FILTER (?genre = false || ?genre = true)
    BIND("Referee" AS ?personType).
  }
  UNION
  {
    ?person a foot:ManagerIdentity;
            foot:managerIsBorn ?birthLocation.
    ?person foot:managerFirstName ?personFirstName;
            foot:managerLastName ?personLastName;
            foot:managerDateOfBirth ?dateOfBirth;
            foot:managerGenre ?genre.
    ?birthLocation a foot:Location;
                   foot:cityName ?birthLocationName.
    ?person foot:managerComesFrom ?country.
    ?country foot:countryName ?countryName.
    ?country foot:isIncludedIn ?qualificationZone.
    ?qualificationZone foot:qualificationZoneName ?qualificationZoneName.  
    FILTER (?genre = false || ?genre = true)
    BIND("Manager" AS ?personType).
  }
}
""")

res_as_csv = res.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,personFirstName,personLastName,dateOfBirth,genre,birthLocationName,countryName,qualificationZoneName,personType
0,Daniel,Gomez,1994-08-17,True,Manila,Ivory Coast,AFCON (Africa),Player
1,Mohamed,Salah,1992-06-15,True,Mexico City,Mexico,CONCACAF (North America),Player
2,Hector,Bellerin,1995-03-19,True,Tehran,Iran,AFC (Asia),Player
3,Emma,Smith,1992-08-20,False,Madrid,France,UEFA (Europe),Player
4,Olivia,Martini,1995-06-22,False,Rome,Italy,UEFA (Europe),Player
...,...,...,...,...,...,...,...,...
257,Luca,Ricci,1976-03-26,True,Moscow,Russia,UEFA (Europe),Manager
258,Maurizio,Sarri,1959-01-10,True,Canberra,Australia,OFC (Oceania ),Manager
259,Ole,Gunnar Solskjaer,1973-02-26,True,Rio de Janeiro,Brazil,CONMONBOL (South America),Manager
260,Sebastian,Wagner,1975-06-10,True,Brussels,Belgium,UEFA (Europe),Manager


4.b. The number of players, referees and managers.

In [9]:
res = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>

SELECT ?personType (COUNT(?person) as ?count)
WHERE {
  {
    ?person a foot:PlayerIdentity.
    BIND("Player" AS ?personType).
  }
  UNION
  {
    ?person a foot:RefereeIdentity.
    BIND("Referee" AS ?personType).
  }
  UNION
  {
    ?person a foot:ManagerIdentity.
    BIND("Manager" AS ?personType).
  }
}
GROUP BY ?personType
""")

res_as_csv = res.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,personType,count
0,Player,84
1,Referee,100
2,Manager,78


5. Determining Team, its manager and some statistics obtained.

In [10]:
res = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT ?teamName ?managerName ?teamFairPlayerDate ?goalsFor ?goalsAgainst  WHERE {
  ?team foot:isManagedBy ?manager.
  ?team a foot:Team;
        foot:teamName ?teamName.
  ?manager a foot:ManagerIdentity;
           foot:managerFirstName ?managerName.
  ?statistics foot:obtainedBy ?team.
  ?statistics a foot:TeamFairPlayIndex;
              foot:teamFairPlayerDate ?teamFairPlayerDate;
              foot:goalsFor ?goalsFor; 
              foot:goalsAgainst ?goalsAgainst.
  ?statistics a foot:TeamFairPlayIndex;
              foot:goalsFor ?goalsFor; 
              foot:goalsAgainst ?goalsAgainst.         
}
""")

res_as_csv = res.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,teamName,managerName,teamFairPlayerDate,goalsFor,goalsAgainst
0,Quito Quails,Sophia,2023-06-01,5,5
1,Quito Quails,Maurizio,2023-06-01,5,5
2,Antananarivo Tigers,Antonio,2023-06-01,9,6
3,Antananarivo Tigers,Lea,2023-06-01,9,6
4,Suva Stingrays,Maja,2023-06-01,6,4
...,...,...,...,...,...
61,Port Moresby Penguins,Hannah,2023-06-01,4,true
62,Port Moresby Penguins,Hannah,2023-06-01,4,1
63,Port Moresby Penguins,Ernesto,2023-06-01,4,1
64,Port Moresby Penguins,Ernesto,2023-06-01,4,true


Determining the players who have as an employer Dublin Dragons.

In [11]:
res = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT  ?playerLastName ?teamName WHERE {
  ?team foot:playerEmployer ?player.
  ?player a foot:PlayerIdentity;
           foot:playerLastName ?playerLastName.       
  ?team a foot:Team; 
          foot:teamName ?teamName.
  FILTER(?teamName = "Dublin Dragons").
}
""")

res_as_csv = res.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,playerLastName,teamName
0,Johnson,Dublin Dragons
1,Ibrahimovic,Dublin Dragons


African or European countries and its associated qualification zone.

In [12]:
res = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT ?countryName ?qualificationZoneName WHERE {
  ?continent foot:isMadeUp ?country.
  ?continent a foot:QualificationZone;
             foot:qualificationZoneName ?qualificationZoneName.      
  ?country a foot:OriginCountry; 
           foot:countryName ?countryName.
  FILTER(?qualificationZoneName = "AFCON (Africa)" || ?qualificationZoneName = "UEFA (Europe)").
}
""")

res_as_csv = res.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,countryName,qualificationZoneName
0,Guinea,AFCON (Africa)
1,Morocco,AFCON (Africa)
2,Ethiopia,AFCON (Africa)
3,Zimbabwe,AFCON (Africa)
4,Mali,AFCON (Africa)
5,Ivory Coast,AFCON (Africa)
6,Sierra Leone,AFCON (Africa)
7,Congo,AFCON (Africa)
8,Tanzania,AFCON (Africa)
9,Angola,AFCON (Africa)


Manager coaches player and the team he manages.

In [13]:
fix = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT ?managerFirstName ?managerLastName ?playerFirstName ?playerLastName ?teamName WHERE {
  ?manager foot:coaches ?player.
  ?manager a foot:ManagerIdentity;
           foot:managerFirstName ?managerFirstName;
           foot:managerLastName ?managerLastName.
  ?player a foot:PlayerIdentity; 
         foot:playerFirstName ?playerFirstName;
         foot:playerLastName ?playerLastName.
  ?manager foot:manages ?team.
  ?team a foot:Team; 
        foot:teamName ?teamName.
}
ORDER BY ?managerFirstName
""")

res_as_csv = fix.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,managerFirstName,managerLastName,playerFirstName,playerLastName,teamName
0,Alessandro,Rossi,Sergio,Aguero,Hanoi Hornets
1,Alessandro,Rossi,Sarah,Brown,Hanoi Hornets
2,Alessandro,Wagner,Sophia,Kim,Toronto Titans
3,Andreas,Weber,Michael,Johnson,Helsinki Huskies
4,Andreas,Fischer,Mohamed,Salah,Luanda Leopards
...,...,...,...,...,...
104,Sophie,Lehmann,Benjamin,Dupont,Port Vila Panthers
105,Stefan,Mayer,Daniel,Gomez,Georgetown Gladiators
106,Stefan,Mayer,Antoine,Griezmann,Georgetown Gladiators
107,Tim,Müller,James,Lee,Harare Hawks


Player is coached by manager.

In [14]:
fix = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT ?playerFirstName ?playerLastName ?managerFirstName ?managerLastName ?teamName WHERE {
  ?player foot:isCoachedBy ?manager.
  ?manager a foot:ManagerIdentity;
           foot:managerFirstName ?managerFirstName;
           foot:managerLastName ?managerLastName.
  ?player a foot:PlayerIdentity; 
         foot:playerFirstName ?playerFirstName;
         foot:playerLastName ?playerLastName.
  ?manager foot:manages ?team.
  ?team a foot:Team; 
        foot:teamName ?teamName.
}
""")

res_as_csv = fix.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,playerFirstName,playerLastName,managerFirstName,managerLastName,teamName
0,James,Lee,Fabio,Keller,Mexico City Mavericks
1,Vinicius,Junior,Sophia,Schulz,Port Vila Panthers
2,Benjamin,Dupont,Sophia,Schulz,Port Vila Panthers
3,Maria,Garcia,Martin,Fischer,Tehran Tigers
4,Chloe,Johnson,Mia,Wagner,San José Sharks
...,...,...,...,...,...
104,Lucas,Kim,Ole,Gunnar Solskjaer,Guatemala City Griffins
105,Paulo,Dybala,Sebastian,Wagner,Bangkok Buccaneers
106,Thomas,Nguyen,Sebastian,Wagner,Bangkok Buccaneers
107,Thiago,Silva,Emilia,Hofmann,Montevideo Mavericks


Jurgen Klopp coaches player.

In [15]:
fix = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT ?playerFirstName ?playerLastName ?managerFirstName ?managerLastName ?teamName WHERE {
  ?manager foot:coaches ?player.
  ?manager a foot:ManagerIdentity;
           foot:managerFirstName ?managerFirstName;
           foot:managerLastName ?managerLastName.
  ?player a foot:PlayerIdentity; 
         foot:playerFirstName ?playerFirstName;
         foot:playerLastName ?playerLastName.
  ?manager foot:manages ?team.
  ?team a foot:Team; 
        foot:teamName ?teamName.
  FILTER(?managerFirstName = "Jurgen")
}
""")

res_as_csv = fix.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,playerFirstName,playerLastName,managerFirstName,managerLastName,teamName
0,Thomas,Nguyen,Jurgen,Klopp,Addis Ababa Alligators


Player plays at home.

In [16]:
fix = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT ?playerFirstName ?playerLastName ?stadiumName WHERE {
  ?player foot:playsAtHome ?stadium.
  ?player a foot:PlayerIdentity;
         foot:playerLastName ?playerLastName;
         foot:playerFirstName ?playerFirstName.  
  ?stadium a foot:SportsFacility; 
           foot:stadiumName ?stadiumName.
}
""")

res_as_csv = fix.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,playerFirstName,playerLastName,stadiumName
0,Daniel,Gomez,Brentford Community Stadium
1,Mohamed,Salah,Turf Moor
2,Hector,Bellerin,KCOM Stadium
3,Emma,Smith,Anfield
4,Olivia,Martini,Estadio Ramon Sanchez Pizjuan
...,...,...,...
79,Neymar,Jr.,Etihad Stadium
80,Marta,Vieira,Old Trafford
81,Erling,Haaland,Bloomfield Road
82,David,Silva,Molineux Stadium


Determine where a player attends in the qualification zone.

In [17]:
fix = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT ?playerFirstName ?playerLastName ?name WHERE {
  ?player foot:playerAttendsIn ?zone.
  ?player a foot:PlayerIdentity;
         foot:playerLastName ?playerLastName;
         foot:playerFirstName ?playerFirstName.  
  ?zone a foot:QualificationZone; 
           foot:qualificationZoneName ?name.
}
""")

res_as_csv = fix.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,playerFirstName,playerLastName,name
0,Daniel,Gomez,AFCON (Africa)
1,Mohamed,Salah,CONCACAF (North America)
2,Hector,Bellerin,AFC (Asia)
3,Emma,Smith,UEFA (Europe)
4,Olivia,Martini,UEFA (Europe)
...,...,...,...
79,Neymar,Jr.,CONCACAF (North America)
80,Marta,Vieira,AFCON (Africa)
81,Erling,Haaland,OFC (Oceania )
82,David,Silva,AFC (Asia)


Determine where a referee attends in the qualification zone.

In [18]:
fix = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT ?refereeFirstName ?refereeLastName ?name WHERE {
  ?player foot:refereeAttendsIn ?zone.
  ?player a foot:RefereeIdentity;
         foot:refereeLastName ?refereeLastName;
         foot:refereeFirstName ?refereeFirstName.  
  ?zone a foot:QualificationZone; 
           foot:qualificationZoneName ?name.
}
""")

res_as_csv = fix.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,refereeFirstName,refereeLastName,name
0,Charlotte,Rivera,AFCON (Africa)
1,Ethan,Wilson,AFCON (Africa)
2,Aria,Hall,CONMONBOL (South America)
3,William,Thompson,AFCON (Africa)
4,Anna,Torres,AFC (Asia)
...,...,...,...
95,David,Martinez,AFCON (Africa)
96,Isabella,Young,AFCON (Africa)
97,Matthew,Reed,CONMONBOL (South America)
98,Ella,White,CONMONBOL (South America)


Determine where a manager attends in the qualification zone.

In [19]:
fix = g.query("""
PREFIX foot: <http://www.example.org/foot-infos/ont#>
SELECT ?managerFirstName ?managerLastName ?name WHERE {
  ?manager foot:managerAttendsIn ?zone.
  ?manager a foot:ManagerIdentity;
         foot:managerLastName ?managerLastName;
         foot:managerFirstName ?managerFirstName.  
  ?zone a foot:QualificationZone; 
           foot:qualificationZoneName ?name.
}
""")

res_as_csv = fix.serialize(format="csv")
df = pd.read_csv(BytesIO(res_as_csv))
df

,managerFirstName,managerLastName,name
0,Fabio,Keller,AFC (Asia)
1,Sophia,Schulz,AFC (Asia)
2,Martin,Fischer,CONMONBOL (South America)
3,Mia,Wagner,OFC (Oceania )
4,Hannah,Bauer,CONCACAF (North America)
...,...,...,...
73,Luca,Ricci,UEFA (Europe)
74,Maurizio,Sarri,OFC (Oceania )
75,Ole,Gunnar Solskjaer,CONMONBOL (South America)
76,Sebastian,Wagner,UEFA (Europe)
